In [45]:
import pandas as pd
import numpy as np

%pylab inline

gtd_df = pd.read_excel('附件1.xlsx')

gtd_df.info() # 在对数据进行处理之前应该先查看加载数据的相关信息

Populating the interactive namespace from numpy and matplotlib
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114183 entries, 0 to 114182
Columns: 135 entries, eventid to related
dtypes: datetime64[ns](1), float64(54), int64(23), object(57)
memory usage: 117.6+ MB


In [2]:
# total no of columns and rows present in data
print("数据的尺寸:",gtd_df.shape)

# Check the number of missing values in each attribute
count = gtd_df.isnull().sum()
percent = round(count / gtd_df.shape[0] * 100, 2)
series = [count, percent]
result = pd.concat(series, axis=1, keys=['Count','Percent'])
result.sort_values(by='Count', ascending=False)  # 按缺失值数量倒序

数据的尺寸: (114183, 135)


,Count,Percent
weaptype4,114178,100.00
weaptype4_txt,114178,100.00
weapsubtype4,114177,99.99
weapsubtype4_txt,114177,99.99
gsubname3,114163,99.98
claimmode3,114058,99.89
claimmode3_txt,114058,99.89
gsubname2,114029,99.87
divert,114026,99.86
kidhijcountry,113971,99.81


In [3]:
target_attrs = result[result['Percent'] < 90.0]
print('留下来的特征所占比例:', round(target_attrs.shape[0]/result.shape[0], 2))
df_50_90 = target_attrs[target_attrs['Percent'] > 50.0]
round(df_50_90.shape[0]/target_attrs.shape[0], 2)  # 特征数所占比例

留下来的特征所占比例: 0.53


0.19

In [4]:
a = result.query('0<Percent<0.1')
a

,Count,Percent
provstate,10,0.01
specificity,6,0.01
scite1,61,0.05


In [5]:
b = result.query('0<Count<100')
b

,Count,Percent
provstate,10,0.01
specificity,6,0.01
doubtterr,1,0.00
multiple,1,0.00
ishostkid,3,0.00
scite1,61,0.05


由于数据容量比较大, 所以, 我们可以直接扔掉 包含特征: `'doubtterr', 'multiple', 'provstate', 'scite1', 'ishostkid', 'specificity'` 的含缺失值的样本行.

In [6]:
attrs = []
for attr in b.index:
    attrs.extend(gtd_df[attr][gtd_df[attr].isna()].index.tolist())

In [7]:
df = gtd_df.loc[list(set(gtd_df.index) - set(attrs)), :]

In [8]:
df.shape

(114101, 135)

In [9]:
np.unique(gtd_df['doubtterr'].isin(['NaN']))

array([False,  True])

In [10]:
np.unique(df['doubtterr'].isin(['NaN']))

array([False])

也可:
    
```py
# Removing columns which has 80% null values
def remove_columns_missing_values(data, min_threshold):
    for col in data.columns:
        rate = data[col].isnull().sum()/float(len(data)) * 100
        if rate >= min_threshold:
            data = data.drop(col, 1)
    return data

data = remove_columns_missing_values(df , 80)
print("可以得到的特征 :",len(data.columns))
```

Pandas 中进行数据类型转换有三种基本方法：

- 使用 `astype()` 函数进行强制类型转换
- 自定义函数进行数据类型转换
- 使用 Pandas 提供的函数如 `to_numeric()`、`to_datetime()`

注意: 从上面两个例子可以看出，当待转换列中含有不能转换的特殊值时(例子中￥,ErrorValue等)astype()函数将失效。有些时候 `astype()` 函数执行成功了也并不一定代表着执行结果符合预期(神坑!）

参考: 

- [Pandas数据类型转换的几个小技巧](https://segmentfault.com/a/1190000014713098)
- [【原】十分钟搞定pandas](https://www.cnblogs.com/chaosimple/p/4153083.html)


## 在Pandas中更改列的数据类型

- `pd.to_datetime` 和 `pd.to_timedelta` 可将数据转换为日期和时间戳。
- `infer_objects()` 方法，用于将具有对象数据类型的 DataFrame 的列转换为更具体的类型。

## 查看数据类型

- `df.dtypes`
- `series.dtype`
- `get_dtype_counts()`

如果一列中含有多个类型,则该列的类型会是 `object`,同样字符串类型的列也会被当成 `object` 类型. 
不同的数据类型也会被当成 `object`,比如 `int32`,`float32`.

In [11]:
df.get_dtype_counts()

float64           54
int64             23
datetime64[ns]     1
object            57
dtype: int64

In [12]:
keep_attrs = target_attrs[target_attrs['Percent']<50.0]

subset_df = df.loc[:, keep_attrs.index.values]

subset_df.get_dtype_counts()

int64      23
object     17
float64    18
dtype: int64

In [13]:
subset_df.shape

(114101, 58)

In [14]:
np.unique(subset_df.isnull().sum())

array([    0,   161,   353,   378,   428,   562,   926,  1115,  2668,
        3363,  4279,  4494,  7003,  8223,  9076, 10313, 13158, 38432],
      dtype=int64)

In [15]:
np.unique(subset_df['doubtterr'].isnull())

array([False])

In [16]:
subset_df[subset_df['multiple'].isnull()]

,eventid,iyear,imonth,iday,extended,country,country_txt,region,region_txt,provstate,...,nwoundte,property,ishostkid,scite1,scite2,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY


### [通过列类型选取列](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.select_dtypes.html#pandas.DataFrame.select_dtypes)

方法: `DataFrame.select_dtypes(include=None, exclude=None)`

参数: `include`, `exclude` : list-like(传入想要查找的类型)

返回: `subset` : DataFrame

In [17]:
feature_names = set(subset_df.columns)

# 数值变量集合
num_feature_names = {
    'nperps', 'nperpcap', 'nkill', 'nkillus', 'nkillter', 'nwound',
    'nwoundus', 'nwoundte', 'propvalue', 'nhostkid', 'nhostkidus',
    'nhours', 'ndays', 'ransomamt', 'ransomamtus', 'ransompaid','ransompaidus', 'nreleased'
}
exta_names = {'eventid','iday','imonth','iyear','latitude', 'longitude'}

txt_names = set(subset_df.select_dtypes(['object']).columns)
txt_names.update({'city', 'summary', 'provstate', 'corp1', 'target1', 'gname','scite1', 'scite2', 'dbsource'})

cat_names = feature_names - num_feature_names - exta_names - txt_names

In [18]:
len(feature_names)

58

In [19]:
subset_df.get_dtype_counts()

int64      23
object     17
float64    18
dtype: int64

### 类别信息

In [20]:
cat_df = subset_df[list(cat_names)]

In [21]:
for att in cat_names:
    s = subset_df[att].isnull().sum()
    if s != 0:
        print(att,  s)

guncertain1 378
targsubtype1 7003
natlty1 1115
weapsubtype1 9076


In [22]:
subset_df['guncertain1'].fillna(-1, inplace=True)
subset_df['natlty1'].fillna(-1, inplace=True)
subset_df['targsubtype1'].fillna(-1, inplace=True)
subset_df.loc[subset_df['weapsubtype1'] == 13, 'weapsubtype1'] = -1  # 13 未知
subset_df['weapsubtype1'].fillna(-1, inplace=True)
subset_df.loc[subset_df['doubtterr'] == -9, 'doubtterr'] = -1  # -9 未知
subset_df['doubtterr'].fillna(-1, inplace=True)
subset_df.loc[subset_df['specificity'] == 5, 'specificity'] = -1
subset_df.loc[subset_df['attacktype1'] == 9, 'attacktype1'] = -1  #  9 未知
subset_df.loc[subset_df['targtype1'] == 20, 'targtype1'] = -1 # 20 未知
subset_df.loc[subset_df['property'] == -9, 'property'] = -1
subset_df.loc[subset_df['ishostkid'] == -9, 'ishostkid'] = -1
subset_df.loc[subset_df['INT_LOG'] == -9, 'INT_LOG'] = -1
subset_df.loc[subset_df['INT_IDEO'] == -9, 'INT_IDEO'] = -1
subset_df.loc[subset_df['INT_MISC'] == -9, 'INT_MISC'] = -1
subset_df.loc[subset_df['INT_ANY'] == -9, 'INT_ANY'] = -1

In [23]:
#subset_df['attacktype1'].replace(9, -1)

subset_df.loc[subset_df['claimed'] == -9, 'claimed'] = -1
subset_df['claimed'].fillna(-1, inplace=True)

subset_df.loc[subset_df['vicinity'] == -9, 'vicinity'] = -1

### 文本变量

In [24]:
for att in txt_names:
    s = subset_df[att].isnull().sum()
    if s != 0:
        print(att, s)

weapsubtype1_txt 9076
targsubtype1_txt 7003
scite2 38432
target1 161
city 428
natlty1_txt 1115
corp1 10313


In [25]:
subset_df['targsubtype1_txt'].fillna('Unknown', inplace=True)
subset_df['corp1'].fillna('Unknown', inplace=True)
subset_df['city'].fillna('Unknown', inplace=True)
subset_df['target1'].fillna('Unknown', inplace=True)
subset_df['weapsubtype1_txt'].fillna('Unknown', inplace=True)
subset_df['natlty1_txt'].fillna('Unknown', inplace=True)
subset_df['scite2'].fillna('Unknown', inplace=True)

### 可计算变量

In [26]:
cal_names = feature_names & num_feature_names

In [27]:
for att in cal_names:
    s = subset_df[att].isnull().sum()
    if s != 0:
        print(att, s)

nperps 13158
nwound 8223
nwoundus 562
nkillter 2668
nkillus 353
nwoundte 4494
nkill 4279
nperpcap 3363


In [28]:
subset_df.loc[subset_df['nperpcap'] == -9, 'nperpcap'] = np.nan
subset_df.loc[subset_df['nperpcap'] == -99, 'nperpcap'] = np.nan
subset_df.loc[subset_df['nperps'] == -9, 'nperps'] = np.nan
subset_df.loc[subset_df['nperps'] == -99, 'nperps'] = np.nan

In [29]:
for att in cal_names:
    s = subset_df[att].isnull().sum()
    if s != 0:
        print(att, s)

nperps 94366
nwound 8223
nwoundus 562
nkillter 2668
nkillus 353
nwoundte 4494
nkill 4279
nperpcap 4227


变量 `nperps=94366` 的缺失值相对较多, 考虑移除该特征:

In [30]:
num_feature_names.remove('nperps')

cal_names.remove('nperps')

feature_names.remove('nperps')

del subset_df['nperps']

### 附加变量

In [31]:
for att in exta_names:
    s = subset_df[att].isnull().sum()
    if s != 0:
        print(att, s)

latitude 926
longitude 926


In [32]:
# Select the observations that contain null
ll_df = subset_df[np.isnan(subset_df.latitude)]
print(ll_df.shape)   # 926 个 NaN 值

(926, 57)


In [33]:
ll_df[['latitude', 'longitude']].isnull().sum()

latitude     926
longitude    926
dtype: int64

In [34]:
934 / subset_df.shape[0]

0.008185730186413791

因而, 可以忽略这 $934$ 个样本.

In [35]:
attrs = []
for attr in exta_names:
    attrs.extend(subset_df[attr][subset_df[attr].isna()].index.tolist())

In [36]:
attr_index = set(subset_df.index) - set(attrs)

In [37]:
gtd_clean = subset_df.loc[attr_index,:]

In [38]:
gtd_clean.isnull().sum()

eventid                0
iyear                  0
imonth                 0
iday                   0
extended               0
country                0
country_txt            0
region                 0
region_txt             0
provstate              0
city                   0
latitude               0
longitude              0
specificity            0
vicinity               0
summary                0
crit1                  0
crit2                  0
crit3                  0
doubtterr              0
multiple               0
success                0
suicide                0
attacktype1            0
attacktype1_txt        0
targtype1              0
targtype1_txt          0
targsubtype1           0
targsubtype1_txt       0
corp1                  0
target1                0
natlty1                0
natlty1_txt            0
gname                  0
guncertain1            0
individual             0
nperpcap            4154
claimed                0
weaptype1              0
weaptype1_txt          0


In [40]:
gtd_clean.get_dtype_counts()

int64      23
object     17
float64    17
dtype: int64

In [41]:
gtd_clean.shape

(113175, 57)

In [42]:
names_dict = {
    'exta_names': exta_names,
    'cat_names': cat_names,
    'txt_names': txt_names,
    'cal_names': cal_names
}

In [43]:
import pickle

with open('./data/gtd_names.json', 'wb') as fp:
    pickle.dump(names_dict, fp)

with open('./data/gtd_names.json', 'rb') as fp:
    s = pickle.load(fp)

In [44]:
gtd_clean.to_excel("./data/gtd_preprocessed.xlsx")   # 写入本地磁盘

#subset_df[cal_names] = subset_df[cal_names].replace(np.nan, -1)